# Execution Modes: Jobs, Sessions, and Batch - Code Laboratory

**Section 4: Running Circuits | Part 3 of 3**

> 📖 **For concepts, analogies, and theory**: See [README.md](README.md)
> 🔬 **This notebook**: Executable code demonstrations and API exploration

---

## 🚀 Quick API Reference

| Mode | Syntax | Use Case | Key Benefit |
|------|--------|----------|-------------|
| **Job** | `Sampler(mode=backend)` | Single circuit, testing | Simplest, no overhead |
| **Batch** | `Batch(backend)` → `Sampler(mode=batch)` | Multiple independent circuits | Parallel execution, grouped |
| **Session** | `Session(backend)` → `Estimator(mode=session)` | Iterative algorithms (VQE/QAOA) | Reserved QPU, no re-queuing |

---

## 📋 Notebook Contents

1. **Setup** - Imports and helper functions
2. **Job Mode** - Single circuit execution
3. **Batch Mode** - Parallel independent circuits
4. **Session Mode** - Sequential with reservation
5. **Mode Comparison** - Side-by-side examples
6. **Decision Tree** - When to use each mode
7. **Trap Demonstrations** - Common mistakes
8. **Code Challenges** - Test your understanding

---

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp

# IBM Quantum Runtime imports
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
    Session,
    Batch
)

# For demonstrations without real hardware
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.primitives import StatevectorSampler, StatevectorEstimator

# Helper: Create test circuits
def create_bell_circuit():
    """Create a Bell state circuit."""
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    return qc

def create_parameterized_circuit(theta):
    """Create a VQE-style parameterized circuit."""
    qc = QuantumCircuit(2)
    qc.ry(theta, 0)
    qc.cx(0, 1)
    return qc

def create_grover_circuit():
    """Create a simple Grover circuit."""
    qc = QuantumCircuit(2, 2)
    qc.h([0, 1])
    qc.cz(0, 1)  # Oracle
    qc.h([0, 1])
    qc.z([0, 1])
    qc.cz(0, 1)
    qc.h([0, 1])
    qc.measure([0, 1], [0, 1])
    return qc

# Test Hamiltonian for Estimator examples
H = SparsePauliOp(["ZZ", "IZ", "ZI"], coeffs=[1.0, 0.5, 0.5])

print("✅ Setup complete - all imports successful!")
print(f"   Test Hamiltonian: {H}")

---

## 2. Job Mode - Single Circuit Execution

### API Reference
```python
sampler = Sampler(mode=backend)
job = sampler.run([circuit])
result = job.result()
```

> 📖 Theory: Job mode is for single circuits or testing. No session overhead, runs immediately.

### When to Use Job Mode
- ✅ Testing a single circuit
- ✅ Debugging
- ✅ One-off measurements
- ✅ Quick prototypes

In [ ]:
# Job Mode - Single Circuit Execution

print("=" * 60)
print("JOB MODE: Single Circuit Test")
print("=" * 60)

# Create a simple Bell state circuit
bell_circuit = create_bell_circuit()
print(f"\n✅ Created Bell circuit:")
print(bell_circuit.draw(output='text'))

# Using local simulator for demonstration
# In real hardware: backend = service.backend('ibm_brisbane')
backend = FakeManilaV2()

# Job Mode: Direct backend execution
sampler = Sampler(mode=backend)

# Run once
bell_circuit_transpiled = transpile(bell_circuit, backend=backend)
job = sampler.run([bell_circuit_transpiled])
print(f"\n📤 Job submitted: {job.job_id()}")
print(f"   Job status: {job.status()}")

# Get results (blocking call)
result = job.result()
print(f"\n✅ Job completed!")

# Access measurement data
pub_result = result[0]
counts = pub_result.data.c.get_counts()
print(f"   Measurement counts: {counts}")

print("\n📋 Use Case: Quick test of a single circuit")
print("   - No session overhead")
print("   - Runs immediately")
print("   - Perfect for debugging")

---

## 3. Batch Mode - Parallel Independent Circuits

### API Reference
```python
with Batch(backend=backend) as batch:
    sampler = Sampler(mode=batch)
    job = sampler.run(circuits)  # All circuits together
    results = job.result()
```

> 📖 Theory: Batch groups independent circuits for efficient parallel execution. Backend optimizes order.

### When to Use Batch Mode
- ✅ Parameter sweeps (no feedback needed)
- ✅ Benchmarking multiple algorithms
- ✅ 10-100 independent circuits
- ✅ Fair comparison (same backend conditions)

In [ ]:
# Batch Mode - Parameter Sweep Example

print("=" * 60)
print("BATCH MODE: Parameter Sweep")
print("=" * 60)

# Scenario: Benchmark 10 circuits with different rotation angles
angles = np.linspace(0, np.pi, 10)
print(f"\n✅ Creating 10 parameterized circuits")
print(f"   Angles: {angles.round(3)}")

# Create circuits for each parameter
circuits = []
for theta in angles:
    qc = QuantumCircuit(2, 2)
    qc.ry(theta, 0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    circuits.append(qc)

print(f"   Total circuits: {len(circuits)}")

# Batch Mode: All circuits submit together
backend = FakeManilaV2()

with Batch(backend=backend) as batch:
    sampler = Sampler(mode=batch)
    
    print(f"\n📤 Submitting all {len(circuits)} circuits in BATCH")
    # Transpile circuits for the backend
    circuits_transpiled = transpile(circuits, backend=backend)
    job = sampler.run(circuits_transpiled)
    print(f"   Job ID: {job.job_id()}")
    
    # All circuits execute in parallel (backend optimizes order)
    results = job.result()
    print(f"\n✅ Batch completed! Processing results...")

# Analyze results
print(f"\n📊 Results for each angle:")
for i, (theta, result) in enumerate(zip(angles, results)):
    counts = result.data.c.get_counts()
    most_common = max(counts, key=counts.get)
    print(f"   θ={theta:.3f}: Most common state = |{most_common}>")

print("\n📋 Batch Advantages:")
print("   ✅ All circuits queued once")
print("   ✅ Backend optimizes execution order")
print("   ✅ Efficient parallel processing")
print("   ✅ Perfect for independent parameter sweeps")

In [ ]:
# Session Mode - Adaptive Algorithm

print("=" * 60)
print("SESSION MODE: Adaptive QAOA")
print("=" * 60)

print("\n📖 Scenario: QAOA with adaptive depth")
print("   Layer 1 → Analyze → Decide Layer 2 depth")

backend = FakeManilaV2()

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    
    print(f"\n🔒 Session started")
    
    # Layer 1: Initial QAOA circuit
    qc_layer1 = QuantumCircuit(2, 2)
    qc_layer1.h([0, 1])
    qc_layer1.rz(0.5, 0)
    qc_layer1.rz(0.5, 1)
    qc_layer1.measure([0, 1], [0, 1])
    
    print(f"\n📤 Layer 1: Running initial QAOA")

    circuits_transpiled = transpile([qc_layer1], backend=backend)
    job1 = sampler.run(circuits_transpiled)
    result1 = job1.result()
    counts1 = result1[0].data.c.get_counts()
    
    print(f"   Layer 1 counts: {counts1}")
    
    # Adaptive decision based on result
    max_count = max(counts1.values())
    total_shots = sum(counts1.values())
    success_rate = max_count / total_shots
    
    print(f"\n🧠 Analyzing Layer 1:")
    print(f"   Success rate: {success_rate:.2%}")
    
    if success_rate > 0.6:
        print(f"   Decision: Add 1 more layer (good convergence)")
        num_extra_layers = 1
    else:
        print(f"   Decision: Add 2 more layers (needs more depth)")
        num_extra_layers = 2
    
    # Layer 2: Adaptive circuit (NO queue wait!)
    qc_layer2 = QuantumCircuit(2, 2)
    qc_layer2.h([0, 1])
    for _ in range(num_extra_layers + 1):
        qc_layer2.rz(0.5, 0)
        qc_layer2.rz(0.5, 1)
        qc_layer2.cx(0, 1)
    qc_layer2.measure([0, 1], [0, 1])
    
    print(f"\n📤 Layer 2: Running adaptive circuit ({num_extra_layers + 1} layers)")
    circuits_transpiled2 = transpile([qc_layer2], backend=backend)
    job2 = sampler.run(circuits_transpiled2)
    result2 = job2.result()
    counts2 = result2[0].data.c.get_counts()
    
    print(f"   Layer 2 counts: {counts2}")
    print(f"\n✅ Session completed!")

print("\n📋 Why Session for Adaptive Algorithms:")
print("   ✅ Layer 2 runs IMMEDIATELY (no queue)")
print("   ✅ Intermediate result informs next circuit")
print("   ✅ Critical for adaptive error correction")

In [ ]:
# Decision Tree Implementation

def recommend_execution_mode(num_circuits, needs_feedback, is_iterative):
    """
    Decision tree for execution mode selection.
    
    Args:
        num_circuits: Number of circuits to run
        needs_feedback: Does each result inform the next circuit?
        is_iterative: Is this part of an optimization loop?
    
    Returns:
        str: Recommended mode with reason
    """
    print("=" * 60)
    print("EXECUTION MODE DECISION TREE")
    print("=" * 60)
    
    print(f"\n📊 Your requirements:")
    print(f"   Circuits: {num_circuits}")
    print(f"   Needs feedback: {needs_feedback}")
    print(f"   Iterative: {is_iterative}")
    
    print(f"\n🤔 Decision process:")
    
    if needs_feedback or is_iterative:
        print(f"   ✅ Results inform next circuit → SESSION MODE")
        print(f"\n💡 Reason:")
        print(f"   - Reserved QPU prevents re-queuing")
        print(f"   - Essential for VQE/QAOA convergence")
        print(f"   - Each iteration runs immediately")
        return "SESSION"
    
    if num_circuits == 1:
        print(f"   ✅ Single circuit → JOB MODE")
        print(f"\n💡 Reason:")
        print(f"   - Simplest approach")
        print(f"   - No session overhead")
        print(f"   - Perfect for testing")
        return "JOB"
    
    if num_circuits > 1:
        print(f"   ✅ Multiple independent circuits → BATCH MODE")
        print(f"\n💡 Reason:")
        print(f"   - Parallel execution")
        print(f"   - Queue once for all circuits")
        print(f"   - Backend optimizes order")
        return "BATCH"
    
    return "JOB"  # default

# Test different scenarios
print("\n" + "=" * 70)
print("SCENARIO TESTING")
print("=" * 70)

scenarios = [
    {"name": "Single Bell circuit test", "num": 1, "feedback": False, "iterative": False},
    {"name": "50 parameter sweep circuits", "num": 50, "feedback": False, "iterative": False},
    {"name": "VQE with 100 iterations", "num": 100, "feedback": True, "iterative": True},
    {"name": "Grover benchmarking (10 circuits)", "num": 10, "feedback": False, "iterative": False},
    {"name": "QAOA adaptive layers", "num": 5, "feedback": True, "iterative": True},
]

for scenario in scenarios:
    print(f"\n{'='*70}")
    print(f"Scenario: {scenario['name']}")
    mode = recommend_execution_mode(
        scenario['num'], 
        scenario['feedback'], 
        scenario['iterative']
    )
    print(f"\n📌 Recommendation: {mode} MODE")

In [ ]:
# TRAP 3: Batch for VQE (requires feedback)

print("=" * 60)
print("⚠️  TRAP 3: Using Batch for VQE Iterations")
print("=" * 60)

backend = FakeManilaV2()

# ❌ WRONG: Batch for VQE (each iteration needs result first)
print("\n❌ WRONG - Batch for VQE:")
print("   Problem: Can't submit iteration 2 until iteration 1 completes")
print("   (Each iteration needs previous result to update params)")

# This doesn't work for VQE because:
# - Iteration 2 params depend on iteration 1 result
# - Can't create all circuits upfront
# - Batch is for INDEPENDENT circuits only

print("\n   VQE Requirement:")
print("   1. Run circuit with params[0]")
print("   2. Wait for result → compute energy")
print("   3. Update params: params[1] = f(energy)")
print("   4. Run circuit with params[1]  ← Depends on step 2!")
print("   5. Repeat...")

print("\n✅ CORRECT - Session for VQE:")
print("   Reserved QPU allows sequential execution without re-queuing")

with Session(backend=backend, max_time="10m") as session:
    estimator = Estimator(mode=session)
    
    theta = 0.5
    print(f"\n   Starting VQE loop:")
    
    for iteration in range(3):
        # Create circuit with current params
        qc = create_parameterized_circuit(theta)
        
        # Transpile the circuit for the backend
        qc_transpiled = transpile(qc, backend=backend)
        
        # Create observable that matches the transpiled circuit's qubit count
        num_qubits = qc_transpiled.num_qubits
        H_extended = SparsePauliOp(["ZZ" + "I"*(num_qubits-2), 
                                     "IZ" + "I"*(num_qubits-2), 
                                     "ZI" + "I"*(num_qubits-2)], 
                                    coeffs=[1.0, 0.5, 0.5])
        
        # Run with Estimator - pass as PUB (circuit, observable)
        job = estimator.run([(qc_transpiled, H_extended)])
        result = job.result()
        energy = result[0].data.evs
        print(f"   Iter {iteration+1}: θ={theta:.3f}, E={energy:.3f} (no queue!)")
        
        # Update params based on result
        theta -= 0.1 * np.sin(theta)

print("\n📋 Rule:")
print("   VQE/QAOA → SESSION (feedback loop)")
print("   Parameter sweep → BATCH (independent)")

---

## ✅ Summary

📖 **See [README.md](README.md) for concepts, visual diagrams, and decision trees**

This CODE LAB covered:

### Three Execution Modes
- **Job Mode** - Single circuit, direct execution
- **Batch Mode** - Multiple independent circuits, parallel
- **Session Mode** - Iterative algorithms, reserved QPU

### When to Use Each
- Job: Testing, debugging, one-off
- Batch: Parameter sweeps, benchmarks, independent
- Session: VQE, QAOA, adaptive algorithms

### Key Syntax (v0.24.0+)
```python
# Job
sampler = Sampler(mode=backend)

# Batch  
with Batch(backend=backend) as batch:
    sampler = Sampler(mode=batch)

# Session
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
```

### Common Traps
- ❌ Using `session=` instead of `mode=`
- ❌ Session for independent circuits (use Batch)
- ❌ Batch for VQE iterations (use Session)

### Decision Rule
1. Results inform next circuit? → **SESSION**
2. Multiple independent circuits? → **BATCH**
3. Single circuit or test? → **JOB**

---

**Mnemonic: JBS**
- **J**ob: Just one
- **B**atch: Bundle together  
- **S**ession: Sequential reserved

In [ ]:
# 🧪 Code Challenges

print("=" * 60)
print("CHALLENGE 1: Mode Selection")
print("=" * 60)
print("\nScenario: Testing 3 different quantum algorithms")
print("          (Deutsch-Jozsa, Grover, Simon)")
print("          No results needed between algorithms")
print("\nQ: Which execution mode?")
print("\nYour answer (think first!):")
print("...")

answer1 = "BATCH"
print(f"\n✅ Answer: {answer1}")
print(f"   Reason: Multiple independent circuits")
print(f"   - No feedback needed")
print(f"   - Can run in parallel")
print(f"   - Queue once for all 3")

print("\n" + "=" * 60)
print("CHALLENGE 2: VQE vs Batch")
print("=" * 60)
print("\nScenario: 200 iterations of VQE")
print("          Each iteration updates parameters based on previous result")
print("\nQ: Why can't we use Batch mode?")
print("\nYour answer:")
print("...")

answer2 = "Iteration N+1 params depend on iteration N result"
print(f"\n✅ Answer: {answer2}")
print(f"   - Can't create all circuits upfront")
print(f"   - Each result must inform next circuit")
print(f"   - Session prevents queue delays between iterations")

print("\n" + "=" * 60)
print("CHALLENGE 3: Syntax Trap")
print("=" * 60)
print("\nQ: What's wrong with this code?")
print("   with Session(backend=backend) as session:")
print("       sampler = Sampler(session=session)")
print("\nYour answer:")
print("...")

answer3 = "Should use mode=session, not session=session"
print(f"\n✅ Answer: {answer3}")
print(f"   Correct syntax (v0.24.0+):")
print(f"       sampler = Sampler(mode=session)")

print("\n" + "=" * 60)
print("CHALLENGE 4: Decision Tree")
print("=" * 60)
print("\nFor each scenario, identify the correct mode:")
scenarios = [
    ("Single circuit debug run", "JOB"),
    ("50 circuits with different theta values", "BATCH"),
    ("QAOA with adaptive depth decisions", "SESSION"),
    ("Benchmarking 10 algorithms side-by-side", "BATCH"),
    ("VQE energy minimization (100 iters)", "SESSION"),
]

print()
for scenario, correct_mode in scenarios:
    print(f"   {scenario}")
    print(f"   → {correct_mode} ✅\n")

---

## 8. Code Challenges

Test your understanding of execution modes:

In [ ]:
# TRAP 2: Wrong mode for use case

print("=" * 60)
print("⚠️  TRAP 2: Using Session for Independent Circuits")
print("=" * 60)

backend = FakeManilaV2()

# ❌ INEFFICIENT: Session for independent circuits
print("\n❌ INEFFICIENT - Session for independent circuits:")
print("   Wastes reserved time waiting for each result")

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    
    circuits = [create_bell_circuit() for _ in range(5)]
    
    print(f"\n   Running {len(circuits)} independent circuits...")
    for i, circuit in enumerate(circuits):
        job = sampler.run([circuit])
        result = job.result()
        print(f"   Circuit {i+1}: Reserved QPU idle while waiting ⏱️")

print("\n✅ EFFICIENT - Batch for independent circuits:")
print("   All circuits execute in parallel")

with Batch(backend=backend) as batch:
    sampler = Sampler(mode=batch)
    
    circuits = [create_bell_circuit() for _ in range(5)]
    job = sampler.run(circuits)
    results = job.result()
    
    print(f"\n   {len(circuits)} circuits submitted together ⚡")
    print(f"   Backend optimizes execution order")
    print(f"   All results: {len(results)} circuits completed")

print("\n📋 Rule:")
print("   Independent circuits → BATCH")
print("   Iterative feedback → SESSION")

In [ ]:
# TRAP 1: Deprecated session= parameter

print("=" * 60)
print("⚠️  TRAP 1: Deprecated Parameter Syntax")
print("=" * 60)

backend = FakeManilaV2()

# ❌ WRONG: Old syntax (pre-v0.24.0)
print("\n❌ WRONG (deprecated):")
print("   with Session(backend=backend) as session:")
print("       estimator = Estimator(session=session)  # OLD!")

# ✅ CORRECT: New syntax (v0.24.0+)
print("\n✅ CORRECT (v0.24.0+):")
print("   with Session(backend=backend) as session:")
print("       estimator = Estimator(mode=session)  # NEW!")

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    print(f"\n✅ Estimator created with mode=session")
    print(f"   This is the correct v0.24.0+ syntax!")

print("\n📋 Key Change:")
print("   Old: session= parameter")
print("   New: mode= parameter")
print("   Affects: All Primitives (Sampler, Estimator)")

---

## 7. Trap Demonstrations - Common Mistakes

### ⚠️ Trap Topics
1. Using deprecated `session=` parameter
2. Using Session for independent circuits
3. Using Batch for VQE iterations
4. Not understanding mode selection

---

## 6. Decision Tree - When to Use Each Mode

Visual decision guide:

In [ ]:
# Mode Comparison - All Three Modes

print("=" * 70)
print("MODE COMPARISON: Job vs Batch vs Session")
print("=" * 70)

backend = FakeManilaV2()
test_circuit = create_bell_circuit()

# === JOB MODE ===
print("\n1️⃣  JOB MODE (Single Circuit)")
print("-" * 50)

sampler_job = Sampler(mode=backend)
job = sampler_job.run([test_circuit])
result_job = job.result()

print(f"   Execution: Direct backend call")
print(f"   Queue: Once")
print(f"   Result: {result_job[0].data.meas.get_counts()}")
print(f"   ✅ Best for: Testing, debugging, one-off")

# === BATCH MODE ===
print("\n2️⃣  BATCH MODE (Multiple Independent)")
print("-" * 50)

circuits_batch = [test_circuit for _ in range(5)]

with Batch(backend=backend) as batch:
    sampler_batch = Sampler(mode=batch)
    job = sampler_batch.run(circuits_batch)
    results_batch = job.result()

print(f"   Execution: Parallel group")
print(f"   Queue: Once for all {len(circuits_batch)} circuits")
print(f"   Results: {len(results_batch)} circuits completed")
print(f"   ✅ Best for: Parameter sweeps, benchmarks")

# === SESSION MODE ===
print("\n3️⃣  SESSION MODE (Sequential Reserved)")
print("-" * 50)

with Session(backend=backend) as session:
    sampler_session = Sampler(mode=session)
    
    results_session = []
    for i in range(3):
        job = sampler_session.run([test_circuit])
        result = job.result()
        results_session.append(result)
        print(f"   Iteration {i+1}: No re-queue (reserved QPU)")

print(f"   Execution: Sequential with reservation")
print(f"   Queue: Once, then reserved access")
print(f"   Results: {len(results_session)} iterations")
print(f"   ✅ Best for: VQE, QAOA, adaptive algorithms")

# Summary Table
print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)
print(f"{'Mode':<12} {'Queue Wait':<15} {'Use Case':<25} {'Circuits':<15}")
print("-" * 70)
print(f"{'Job':<12} {'Once':<15} {'Single test':<25} {'1':<15}")
print(f"{'Batch':<12} {'Once':<15} {'Independent parallel':<25} {'Many':<15}")
print(f"{'Session':<12} {'Once':<15} {'Iterative feedback':<25} {'Sequential':<15}")

---

## 5. Mode Comparison - Side by Side

Let's compare all three modes executing similar tasks:

In [ ]:
# Session Mode - VQE Simulation

print("=" * 60)
print("SESSION MODE: VQE Iteration Loop")
print("=" * 60)

# VQE parameters
num_iterations = 10
learning_rate = 0.1
initial_theta = 0.5

print(f"\n⚙️  VQE Configuration:")
print(f"   Iterations: {num_iterations}")
print(f"   Initial θ: {initial_theta}")
print(f"   Hamiltonian: {H}")

# Session Mode: Reserved QPU access
backend = FakeManilaV2()

with Session(backend=backend, max_time="30m") as session:
    estimator = Estimator(mode=session)
    
    print(f"\n🔒 Session started - QPU RESERVED")
    print(f"   Max time: 30 minutes")
    
    theta = initial_theta
    energies = []
    
    for iteration in range(num_iterations):
        # Create parameterized circuit
        qc = create_parameterized_circuit(theta)
        
        # Measure expectation value with Estimator
        pub = (qc, H, [theta])
        job = estimator.run([pub])
        result = job.result()
        
        # Extract energy
        energy = result[0].data.evs[0]
        energies.append(energy)
        
        print(f"   Iteration {iteration+1:2d}: θ={theta:.4f}, E={energy:.4f}")
        
        # Classical optimizer update (gradient descent)
        # In real VQE: use scipy.optimize or similar
        gradient = 0.1 * np.sin(theta)  # Simplified
        theta -= learning_rate * gradient
    
    print(f"\n✅ Session completed!")
    print(f"   Final θ: {theta:.4f}")
    print(f"   Final E: {energies[-1]:.4f}")
    print(f"   Energy improvement: {energies[0] - energies[-1]:.4f}")

print("\n📋 Session Advantages:")
print("   ✅ No re-queuing between iterations")
print("   ✅ Reserved QPU (guaranteed access)")
print("   ✅ Each result informs next circuit")
print("   ✅ Essential for VQE/QAOA convergence")

---

## 4. Session Mode - Sequential with Reservation

### API Reference
```python
with Session(backend=backend, max_time="1h") as session:
    estimator = Estimator(mode=session)
    for iteration in range(num_iters):
        job = estimator.run([(circuit, observable, params)])
        result = job.result()
        # Update params based on result
```

> 📖 Theory: Session reserves dedicated QPU access. Perfect for iterative algorithms where each result informs the next circuit.

### When to Use Session Mode
- ✅ VQE (Variational Quantum Eigensolver)
- ✅ QAOA (Quantum Approximate Optimization)
- ✅ Any algorithm with classical feedback loop
- ✅ Adaptive circuits based on intermediate results

In [ ]:
# Batch Mode - Benchmarking Multiple Algorithms

print("=" * 60)
print("BATCH MODE: Algorithm Comparison")
print("=" * 60)

# Create different algorithm circuits
algorithms = {
    'Bell State': create_bell_circuit(),
    'Grover': create_grover_circuit(),
    'GHZ State': QuantumCircuit(3, 3)
}

# Build GHZ circuit
algorithms['GHZ State'].h(0)
algorithms['GHZ State'].cx(0, 1)
algorithms['GHZ State'].cx(0, 2)
algorithms['GHZ State'].measure([0, 1, 2], [0, 1, 2])

print(f"\n✅ Created {len(algorithms)} different algorithms")
for name in algorithms:
    print(f"   - {name}")

# Batch execution for fair comparison
backend = FakeManilaV2()
circuits_list = list(algorithms.values())

with Batch(backend=backend) as batch:
    sampler = Sampler(mode=batch)
    
    print(f"\n📤 Batch submission: All {len(circuits_list)} algorithms")
    job = sampler.run(circuits_list)
    results = job.result()
    print(f"✅ Batch completed!")

# Compare results
print(f"\n📊 Algorithm Comparison:")
for (name, circuit), result in zip(algorithms.items(), results):
    counts = result.data.meas.get_counts()
    print(f"\n{name}:")
    print(f"  Top 3 outcomes: {dict(list(counts.items())[:3])}")

print("\n📋 Why Batch for Benchmarking:")
print("   ✅ Same backend conditions (fair comparison)")
print("   ✅ Single queue wait")
print("   ✅ Efficient resource usage")

---

## 3. Batch Mode - Parallel Independent Circuits

### API Reference
```python
with Batch(backend=backend) as batch:
    sampler = Sampler(mode=batch)
    job = sampler.run(circuits)  # All circuits together
    results = job.result()
```

> 📖 Theory: Batch groups independent circuits for efficient parallel execution. Backend optimizes order.

### When to Use Batch Mode
- ✅ Parameter sweeps (no feedback needed)
- ✅ Benchmarking multiple algorithms
- ✅ 10-100 independent circuits
- ✅ Fair comparison (same backend conditions)